In [30]:
%matplotlib inline
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix


import pandas as pd
from sklearn.metrics import f1_score
data=pd.read_csv('train_covid.csv')

"""
mean_hh_cli_ct                                            4784 non-null   float64
pct_worried_finances                                      0 non-null      float64
pct_cmnty_cli                                             4804 non-null   float64

"""
data=data.drop(["date","pct_worried_finances"],axis=1)


data=data[3500:]

length=data.shape[1]
X=data.iloc[:,0:length-1]
y=data.iloc[:,length-1]
X

,state_code,gender,age_bucket,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,...,pct_diabetes_weighted,pct_cancer_weighted,pct_heart_disease_weighted,pct_high_blood_pressure_weighted,pct_asthma_weighted,pct_chronic_lung_disease_weighted,pct_kidney_disease_weighted,pct_autoimmune_disorder_weighted,pct_no_above_medical_conditions_weighted,pct_multiple_medical_conditions_weighted
3500,wa,female,18-34,243,6.999273e+05,0.4202,0.4202,2.5210,0.8264,20.8333,...,1.9657,1.7569,1.0938,3.8400,18.0154,0.7665,0.0000,8.5529,67.2251,4.1039
3501,wa,male,35-54,221,9.333702e+05,0.0000,0.0000,0.4651,0.0000,11.9266,...,7.4837,0.8460,3.5194,24.6384,13.7735,2.0471,1.2742,3.8096,59.2292,10.9842
3502,wi,female,35-54,278,5.974764e+05,0.0000,0.0000,0.7273,0.0000,19.6364,...,5.7598,2.6474,0.4566,21.0570,19.9029,1.8747,1.4859,12.4038,55.4313,13.6908
3503,wi,male,35-54,153,6.314853e+05,1.3158,0.6579,1.3158,1.9737,14.6667,...,11.6943,2.9089,1.8937,30.5510,18.2160,2.5802,0.7820,4.6942,45.9688,12.5762
3504,al,female,35-54,210,5.006996e+05,3.8278,3.8278,7.1770,3.3654,41.4634,...,16.5946,5.1329,3.7409,34.9700,17.2476,3.9123,1.3839,10.7109,44.3094,24.0436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,tx,male,18-34,202,2.137299e+06,0.5076,0.5076,1.0152,1.0000,14.8936,...,1.4433,0.0000,0.8343,14.5064,17.1602,0.0000,0.7783,1.2843,68.6299,5.5732
4996,wa,female,55+,241,8.556169e+05,0.0000,0.0000,0.8368,0.4149,7.9167,...,19.7887,12.0977,8.5314,40.8167,21.9591,6.3738,2.2981,9.6818,31.5864,33.8810
4997,wa,male,18-34,80,6.770966e+05,0.0000,0.0000,2.5641,0.0000,6.4103,...,2.3666,0.0000,0.0000,3.6700,7.5808,0.0000,0.0000,1.5641,87.1591,3.2100
4998,wa,male,55+,137,8.467731e+05,0.0000,0.0000,0.7353,0.0000,8.9552,...,23.3272,6.0021,15.2388,53.9625,5.2448,7.6780,4.4640,4.3738,30.4135,33.7725


In [31]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
#將是type 是object的取出來:categorical_columns取出來的index


In [32]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [33]:
from sklearn.preprocessing import OneHotEncoder
#onehotencoder使用 -> fit -> transform -> return array 
enc =OneHotEncoder(sparse=False)
enc.fit(X[categorical_columns])
data_encoded=enc.transform(X[categorical_columns])
#產生columns index的名稱
columns_encoded = enc.get_feature_names(categorical_columns)
X_categorical_columns=pd.DataFrame(data_encoded, columns=columns_encoded)

X_withoutcategorical_columns=X.drop(categorical_columns,axis=1)

X=pd.concat([X_withoutcategorical_columns, X_categorical_columns], axis=1)
#onehotencoder之後

X

,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,pct_hh_fever,pct_hh_sore_throat,pct_hh_cough,...,state_code_va,state_code_vt,state_code_wa,state_code_wi,state_code_wv,gender_female,gender_male,age_bucket_18-34,age_bucket_35-54,age_bucket_55+
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,202.0,2.137299e+06,0.5076,0.5076,1.0152,1.0000,14.8936,1.0000,6.0000,11.8812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4996,241.0,8.556169e+05,0.0000,0.0000,0.8368,0.4149,7.9167,0.4149,6.2241,14.1079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,80.0,6.770966e+05,0.0000,0.0000,2.5641,0.0000,6.4103,0.0000,1.2500,7.5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,137.0,8.467731e+05,0.0000,0.0000,0.7353,0.0000,8.9552,0.0000,0.7299,6.5693,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:

clf = RandomForestClassifier(random_state=0)
X=X[1500:]
X.fillna(0)

,n,weight_sums,pct_cli,pct_ili,pct_cli_anosmia_ageusia,pct_hh_cli,pct_cmnty_cli,pct_hh_fever,pct_hh_sore_throat,pct_hh_cough,...,state_code_va,state_code_vt,state_code_wa,state_code_wi,state_code_wv,gender_female,gender_male,age_bucket_18-34,age_bucket_35-54,age_bucket_55+
3500,243.0,6.999273e+05,0.4202,0.4202,2.5210,0.8264,20.8333,1.2397,10.3306,12.3457,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3501,221.0,9.333702e+05,0.0000,0.0000,0.4651,0.0000,11.9266,0.0000,2.7273,4.5249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3502,278.0,5.974764e+05,0.0000,0.0000,0.7273,0.0000,19.6364,0.3610,9.3525,10.0719,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3503,153.0,6.314853e+05,1.3158,0.6579,1.3158,1.9737,14.6667,1.9737,9.2105,13.7255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3504,210.0,5.006996e+05,3.8278,3.8278,7.1770,3.3654,41.4634,3.3654,11.9617,15.7143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,202.0,2.137299e+06,0.5076,0.5076,1.0152,1.0000,14.8936,1.0000,6.0000,11.8812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,241.0,8.556169e+05,0.0000,0.0000,0.8368,0.4149,7.9167,0.4149,6.2241,14.1079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,80.0,6.770966e+05,0.0000,0.0000,2.5641,0.0000,6.4103,0.0000,1.2500,7.5000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,137.0,8.467731e+05,0.0000,0.0000,0.7353,0.0000,8.9552,0.0000,0.7299,6.5693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# __Make Pipeline__

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
pipe = Pipeline([
                    
                 ('scaler', StandardScaler()),
                ("features",SimpleImputer(strategy='mean')),
                ('min_max', preprocessing.MinMaxScaler()),
                ('feature_select',SelectKBest(chi2)),
                 ('clf',clf)
                  ])
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'features', 'min_max', 'feature_select', 'clf', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'features__add_indicator', 'features__copy', 'features__fill_value', 'features__missing_values', 'features__strategy', 'features__verbose', 'min_max__clip', 'min_max__copy', 'min_max__feature_range', 'feature_select__k', 'feature_select__score_func', 'clf__bootstrap', 'clf__ccp_alpha', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__max_samples', 'clf__min_impurity_decrease', 'clf__min_impurity_split', 'clf__min_samples_leaf', 'clf__min_samples_split', 'clf__min_weight_fraction_leaf', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])

# First Try

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)



c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


0.7093333333333334

In [37]:
#refit
pipe.fit(X, y)

c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


Pipeline(steps=[('scaler', StandardScaler()), ('features', SimpleImputer()),
                ('min_max', MinMaxScaler()),
                ('feature_select',
                 SelectKBest(score_func=<function chi2 at 0x0000012D922F5AF0>)),
                ('clf', RandomForestClassifier(random_state=0))])

In [141]:
# Number of trees in random forest
clf__n_estimators=[100]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
clf__max_depth=[2,4]
# 參數篩選
feature_select__k=[20]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

# Method of selecting samples for training each tree
bootstrap = [True, False]
param_grid = { 
    'feature_select__k':feature_select__k,
    'clf__n_estimators': clf__n_estimators,
    'clf__max_features':max_features,
    'clf__max_depth':clf__max_depth,
    'clf__min_samples_split':min_samples_split,
    'clf__min_samples_leaf':min_samples_leaf,
    'clf__n_jobs':[-1],
    'clf__max_features':max_features,
    'clf__bootstrap':bootstrap
    
    
}
gs=GridSearchCV(estimator=pipe,
               param_grid=param_grid,
              
               cv=5,
              )
gs=gs.fit(X,y)

c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\Ian\Python_workspace\pyml\lib\site-packages\sklearn\

In [142]:
print(gs.best_params_)
print(gs.best_score_)

{'clf__bootstrap': False, 'clf__max_depth': 2, 'clf__max_features': 'auto', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 100, 'clf__n_jobs': -1, 'feature_select__k': 20}
0.7013333333333333


In [ ]:
pipe

In [38]:

test=pd.read_csv("test_covid.csv")

"""
pct_worried_finances                                      0 non-null      float64

"""
# test=test.drop(['date', 'state_code', 'gender', 'age_bucket',"mean_hh_cli_ct","pct_worried_finances","pct_cmnty_cli"],axis=1)

test=test.drop(["date","pct_worried_finances"],axis=1)

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(test)

X_test=test.iloc[:,0:test.shape[1]-1]

#onehotencoder使用 -> fit -> transform -> return array 
enc_test =OneHotEncoder(sparse=False)
enc_test.fit(X_test[categorical_columns])
data_encoded=enc_test.transform(X_test[categorical_columns])

#產生columns index的名稱
print(data_encoded.shape)
columns_encoded = enc_test.get_feature_names(categorical_columns)
print(columns_encoded.shape)
X_categorical_columns=pd.DataFrame(data_encoded, columns=columns_encoded)

X_withoutcategorical_columns=X_test.drop(categorical_columns,axis=1)

X_test=pd.concat([X_withoutcategorical_columns, X_categorical_columns], axis=1)
#onehotencoder之後
X_test.head()
X_test.columns
X_test=pd.DataFrame(X_test,columns=X.columns)
X_test=X_test.fillna(0)

(906, 49)
(49,)


In [40]:

# y_test=gs.best_estimator_.predict(X_test)
y_test=pipe.predict(X_test)
col=['test_condition']
df=pd.DataFrame(y_test,columns=col)
df.to_csv('midterm_random_forest_1500.csv', index = True,index_label='id') 

0.57395
